CAPÍTULO 15 - Regularização
-

No Capítulo 14 vimos maneiras de avaliar o desempenho dos modelos. Neste capítulo explorará a regularização: técnica para melhorar o desempenho em dados de teste.

15.2 Por que regularizar ?

In [166]:
import pandas as pd
acs = pd.read_csv('../../data/acs_ny.csv')
print(acs.columns)

Index(['Acres', 'FamilyIncome', 'FamilyType', 'NumBedrooms', 'NumChildren',
       'NumPeople', 'NumRooms', 'NumUnits', 'NumVehicles', 'NumWorkers',
       'OwnRent', 'YearBuilt', 'HouseCosts', 'ElectricBill', 'FoodStamp',
       'HeatingFuel', 'Insurance', 'Language'],
      dtype='object')


vamos criar matrizes de desing usando a patsy

In [167]:
from patsy import dmatrices

response, predictors = dmatrices('FamilyIncome ~ NumBedrooms + NumChildren + NumPeople + NumRooms + NumUnits + NumVehicles +' \
                                 'NumWorkers + OwnRent + YearBuilt + HouseCosts + ElectricBill + FoodStamp + HeatingFuel + Insurance + Language', data=acs)

In [168]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(predictors, response, random_state=0)

Faremos agora a adequação ao nosso modelo linear. Nesse caso, estamos normalizando os dados para comparar os coeficientes ao usar as técnicas de regularização

In [169]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Normalizar os dados e ajustar o modelo
pipeline = make_pipeline(StandardScaler(), LinearRegression())
pipeline.fit(x_train, y_train)

# Agora você pode usar pipeline.predict para fazer previsões
y_pred = pipeline.predict(x_test)
print(y_pred)
print(pipeline)

[[196605.96895481]
 [ 64581.30047506]
 [126166.63884232]
 ...
 [ 46379.99318873]
 [ 69131.10690884]
 [ 38448.93944903]]
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])


15.3 Regressão LASSO

A primeira técnica de regularização se chama LASSO (Least Absolute Shrinkage an Selection Operator), também conhecida como regularização L1

In [170]:
from sklearn.linear_model import Lasso

# Cria um pipeline que normaliza os dados e ajusta um modelo Lasso
lasso_pipeline = make_pipeline(StandardScaler(), Lasso(max_iter=10000))
lasso_pipeline.fit(x_test, y_test)

lasso_model = lasso_pipeline.named_steps['lasso']
coefs_lasso = pd.DataFrame(list(zip(predictors.design_info.column_names, lasso_model.coef_)),
                           columns=['variable', 'coef_lasso'])

print(coefs_lasso)

                       variable    coef_lasso
0                     Intercept      0.000000
1   NumUnits[T.Single attached]   3907.138710
2   NumUnits[T.Single detached]   4181.061704
3           OwnRent[T.Outright]   6475.305290
4             OwnRent[T.Rented]   1222.822149
5        YearBuilt[T.1940-1949]  -7676.574958
6        YearBuilt[T.1950-1959]  -9038.463519
7        YearBuilt[T.1960-1969]  -8177.292117
8        YearBuilt[T.1970-1979]  -7617.118461
9        YearBuilt[T.1980-1989]  -4304.593236
10       YearBuilt[T.1990-1999]  -6667.913780
11       YearBuilt[T.2000-2004]  -3454.510091
12            YearBuilt[T.2005]  -3886.362558
13            YearBuilt[T.2006]  -2128.432825
14            YearBuilt[T.2007]  -2184.841261
15            YearBuilt[T.2008]    312.296743
16            YearBuilt[T.2009]  -1162.746105
17            YearBuilt[T.2010]   -356.493979
18     YearBuilt[T.Before 1939] -12134.894031
19             FoodStamp[T.Yes]  -5227.340837
20   HeatingFuel[T.Electricity]   

In [171]:
#podemos observar as pontuações de nosso modelo
print(lasso_pipeline.score(x_train, y_train))

0.3322083235465969


In [172]:
print(lasso_pipeline.score(x_test, y_test))

0.3239433914020816


In [173]:
# Fazendo previsões com o modelo Lasso
y_pred_lasso = lasso_pipeline.predict(x_test)

15.4 Regressão de ridge

Analisaremos agora outra técnica de regularização, a regressão ridge, também conhecida como regularização L2

In [174]:
from sklearn.linear_model import Ridge

# Cria um pipeline que normaliza os dados e ajusta um modelo Ridge
ridge_pipeline = make_pipeline(StandardScaler(), Ridge())
ridge_pipeline.fit(x_train, y_train)

ridge_model = ridge_pipeline.named_steps['ridge']

In [175]:
coefs_ridge = pd.DataFrame(
    list(zip(predictors.design_info.column_names, ridge_model.coef_[0])),
    columns=['variable', 'coefs_ridge']
)

model_coefs = pd.DataFrame()
model_coefs = pd.merge(coefs_lasso, coefs_ridge)

print(model_coefs)

                       variable    coef_lasso   coefs_ridge
0                     Intercept      0.000000      0.000000
1   NumUnits[T.Single attached]   3907.138710   2821.231114
2   NumUnits[T.Single detached]   4181.061704   3020.257041
3           OwnRent[T.Outright]   6475.305290   4161.671555
4             OwnRent[T.Rented]   1222.822149    896.856302
5        YearBuilt[T.1940-1949]  -7676.574958    260.838022
6        YearBuilt[T.1950-1959]  -9038.463519   2088.171681
7        YearBuilt[T.1960-1969]  -8177.292117   1748.553203
8        YearBuilt[T.1970-1979]  -7617.118461   1631.804938
9        YearBuilt[T.1980-1989]  -4304.593236   3524.145637
10       YearBuilt[T.1990-1999]  -6667.913780   4822.455119
11       YearBuilt[T.2000-2004]  -3454.510091   2679.042188
12            YearBuilt[T.2005]  -3886.362558   2388.374395
13            YearBuilt[T.2006]  -2128.432825   1503.906730
14            YearBuilt[T.2007]  -2184.841261   1564.643300
15            YearBuilt[T.2008]    312.2

In [176]:
# Fazendo previsões com o modelo Ridge
y_pred_ridge = ridge_pipeline.predict(x_test)

Outro modo de comparar as duas técnicas 
-


comparação de coeficientes

In [177]:
# Coeficientes do modelo Ridge
ridge_coefs = ridge_pipeline.named_steps['ridge'].coef_

# Coeficientes do modelo Lasso
lasso_coefs = lasso_pipeline.named_steps['lasso'].coef_

# Coeficientes do modelo Linear sem regularização
linear_coefs = pipeline.named_steps['linearregression'].coef_

# Comparando os coeficientes
print("Coeficientes do modelo Ridge:")
print(ridge_coefs)

print("\nCoeficientes do modelo Lasso:")
print(lasso_coefs)

print("\nCoeficientes do modelo Linear sem regularização:")
print(linear_coefs)

Coeficientes do modelo Ridge:
[[ 0.00000000e+00  2.82123111e+03  3.02025704e+03  4.16167156e+03
   8.96856302e+02  2.60838022e+02  2.08817168e+03  1.74855320e+03
   1.63180494e+03  3.52414564e+03  4.82245512e+03  2.67904219e+03
   2.38837439e+03  1.50390673e+03  1.56464330e+03  3.77920800e+02
   7.87992038e+01  2.06066273e+03  2.44653221e+03 -4.84542020e+03
   1.48132537e+03  5.03573446e+03  2.97017226e+03  3.34531216e+02
   1.72403578e+01 -4.45832684e+02  1.04536776e+03  6.85814137e+02
  -6.02243752e+02 -3.04624147e+03  1.49121075e+03  5.34093689e+03
  -1.06438033e+04  1.19374445e+04  8.02894115e+03  1.67907723e+04
   3.17543246e+04  5.21816920e+03  1.89140707e+04]]

Coeficientes do modelo Lasso:
[     0.           3907.1387105    4181.061704     6475.30529002
   1222.82214878  -7676.57495763  -9038.46351919  -8177.29211723
  -7617.11846132  -4304.59323569  -6667.91377971  -3454.51009111
  -3886.36255766  -2128.43282524  -2184.84126054    312.29674319
  -1162.74610508   -356.49397885 

Avaliação de modelos

In [178]:
from sklearn.metrics import mean_squared_error, r2_score

# Avaliação do modelo Ridge
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("MSE do modelo Ridge:", mse_ridge)
print("R² do modelo Ridge:", r2_ridge)

# Avaliação do modelo Lasso
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("MSE do modelo Lasso:", mse_lasso)
print("R² do modelo Lasso:", r2_lasso)

# Avaliação do modelo Linear sem regularização
mse_linear = mean_squared_error(y_test, y_pred)
r2_linear = r2_score(y_test, y_pred)

print("MSE do modelo Linear sem regularização:", mse_linear)
print("R² do modelo Linear sem regularização:", r2_linear)

MSE do modelo Ridge: 6801837108.63442
R² do modelo Ridge: 0.31852434713161437
MSE do modelo Lasso: 6747749400.207784
R² do modelo Lasso: 0.3239433914020816
MSE do modelo Linear sem regularização: 6801866648.475862
R² do modelo Linear sem regularização: 0.31852138753666814


15.5 Rede elástica

É uma combinação de ridge e lasso

15.6 Validação cruzada 

Validação cruzada é uma técnica comummente usada na adequação de modelos. É uma maneira de escolher parâmetros ideais para a regularização.

In [182]:
from sklearn.linear_model import ElasticNetCV

en_cv = ElasticNetCV(cv=5, random_state=42).fit(x_train, y_train)

coefs_en_cv = pd.DataFrame(
    list(zip(predictors.design_info.column_names, en_cv.coef_)),
    columns=['variable', 'coef_en_cv']
)

model_coefs = pd.merge(model_coefs, coefs_en_cv, on='variable')
print(model_coefs)

c:\Users\User\anaconda3\envs\pfe_certo\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                       variable    coef_lasso   coefs_ridge  coef_en_cv_x  \
0                     Intercept      0.000000      0.000000      0.000000   
1   NumUnits[T.Single attached]   3907.138710   2821.231114     -0.000000   
2   NumUnits[T.Single detached]   4181.061704   3020.257041      0.000000   
3           OwnRent[T.Outright]   6475.305290   4161.671555      0.000000   
4             OwnRent[T.Rented]   1222.822149    896.856302     -0.000000   
5        YearBuilt[T.1940-1949]  -7676.574958    260.838022     -0.000000   
6        YearBuilt[T.1950-1959]  -9038.463519   2088.171681     -0.000000   
7        YearBuilt[T.1960-1969]  -8177.292117   1748.553203     -0.000000   
8        YearBuilt[T.1970-1979]  -7617.118461   1631.804938     -0.000000   
9        YearBuilt[T.1980-1989]  -4304.593236   3524.145637      0.000000   
10       YearBuilt[T.1990-1999]  -6667.913780   4822.455119      0.000000   
11       YearBuilt[T.2000-2004]  -3454.510091   2679.042188      0.000000   